In [1]:
nobel_winners = [
    {'category': 'Physics',
    'name': 'Albert Einstein',
    'nationality': 'Swiss',
    'sex': 'male',
    'year': 1921},
    {'category': 'Physics',
    'name': 'Paul Dirac',
    'nationality': 'British',
    'sex': 'male',
    'year': 1933},
    {'category': 'Chemistry',
    'name': 'Marie Curie',
    'nationality': 'Polish',
    'sex': 'female',
    'year': 1911},
]

Working with files manually (without csv module)

In [2]:
cols = nobel_winners[0].keys()
cols.sort()
with open('data/nobel_winners.csv', 'w') as f:
    f.write(','.join(cols) + '\n')
    for o in nobel_winners:
        row = [str(o[col]) for col in cols]
        f.write(','.join(row) + '\n')

Now with csv

In [8]:
import csv

#Write
with open('data/nobel_winners.csv', 'wb') as f:
    fieldnames = nobel_winners[0].keys()
    fieldnames.sort()
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()
    for w in nobel_winners:
        writer.writerow(w)
        
#Read using csv.reader which returns strings
with open('data/nobel_winners.csv') as f:
    reader = csv.reader(f)
    for row in reader:
        print(row)
        
#Read in more convenient python dict form
with open('data/nobel_winners.csv') as f:
    reader = csv.DictReader(f)
    nobel_winners = list(reader)
for w in nobel_winners:
    w['year'] = int(w['year']) #convert string to int
print(nobel_winners)



['category', 'name', 'nationality', 'sex', 'year']
['Physics', 'Albert Einstein', 'Swiss', 'male', '1921']
['Physics', 'Paul Dirac', 'British', 'male', '1933']
['Chemistry', 'Marie Curie', 'Polish', 'female', '1911']
[{'category': 'Physics', 'nationality': 'Swiss', 'year': 1921, 'name': 'Albert Einstein', 'sex': 'male'}, {'category': 'Physics', 'nationality': 'British', 'year': 1933, 'name': 'Paul Dirac', 'sex': 'male'}, {'category': 'Chemistry', 'nationality': 'Polish', 'year': 1911, 'name': 'Marie Curie', 'sex': 'female'}]


Now save in JSON instead of csv

In [12]:
import json

with open('data/nobel_winners.json', 'w') as f:
    json.dump(nobel_winners, f)
    
with open('data/nobel_winners.json') as f:
    nobel_winners = json.load(f)
    #print(f.read()) #note that we don't need to convert year to ints
print(nobel_winners)

[{u'category': u'Physics', u'nationality': u'Swiss', u'year': 1921, u'name': u'Albert Einstein', u'sex': u'male'}, {u'category': u'Physics', u'nationality': u'British', u'year': 1933, u'name': u'Paul Dirac', u'sex': u'male'}, {u'category': u'Chemistry', u'nationality': u'Polish', u'year': 1911, u'name': u'Marie Curie', u'sex': u'female'}]


Dealing with Dates and Times


In [13]:
from datetime import datetime
json.dumps(datetime.now()) # will trigger an error

TypeError: datetime.datetime(2017, 8, 21, 19, 11, 15, 663694) is not JSON serializable

In [14]:
import datetime
from dateutil import parser
import json

class JSONDateTimeEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, (datetime.date, datetime.datetime)):
            return obj.isoformat()
        else:
            return json.JSONEncoder.default(self, obj)
        
def mydumps(obj):
    return json.dumps(obj, cls=JSONDateTimeEncoder)
now_str = mydumps({'time': datetime.datetime.now()})
now_str


'{"time": "2017-08-21T19:15:10.159444"}'

In [17]:
from datetime import datetime
import dateutil

d = datetime.now()
d_iso = d.isoformat()  # convert to text
print(d_iso)

d = dateutil.parser.parse(d_iso) # back to a datetime object
print(d)

2017-08-21T19:31:13.616605
2017-08-21 19:31:13.616605
